# AWS를 활용한 인공지능 모델 배포

## [contents]
### 1. 클라우드 기초
### 2. 실습 : AWS & 실습 환경 세팅
### 3. API to serve ML model
### 4. 실습 : Serialization & De-serialization
### 5. 실습 : Inference를 위한 model handler개발
### 6. Flask 기반 감성분석 API 개발



----
## 1. `클라우드` 기초
- 클라우드 = 인터넷 기반 컴퓨팅의 일종
### 1-1) 클라우드를 `사용하는 이유와 그 예시`
- 인터넷 사용자가 크게 증가하고 다양한 서비스를 제공하게 되면서 필요한 때에 필요한 만큼 서버를 증설하기 원하는 `온디맨드 수요 증가` (클라우드를 통해서 적은 비용으로 빠르게 필요한 IT 환경(빅데이터, 플랫폼, 인고지능 개발 등의 환경 마련 가능)
- ex) `인공지능 서비스 제공` : `도커와 같은 가상화 기술`을 통해 `GPU 활용과 소프트웨어 설치 및 배포 등의 작업에 비용과 시간 절감 가능
- ex) `속도-주문형 셀프 서비스` : 클라우드 제공자와 `별도의 커뮤니케이션 없이` 원하는 클라우드 서비스를 바로 이용 가능
- ex) `접근성` : 사용자의 위치, 시간과 관계없이 어떤 디바이스로도 접근 가능
- ex) `확장성` : 이용량 증가시 유연하게 추가 확장 가능
- ex) `생산성`, `측정 가능성` : 하드웨어, 소프트웨어 설치 비용 절감, 클라우드 서비스를 사용한 만큼만 과금
- ex) `보안`, `안정성` : 클라우드 공급자가 전체적으로 보안이나 안정성 제공

### 1-2) 클라우드 컴퓨팅 `운용 모델`
#### `구축 및 배포 유형`에 따라 세 가지 형태로 구분
#### `Public`, `Private`, `Hybrid` (분산형 등 다양한 형태가 존재함, 최근에는 네이버, 아마존, 마소 등 여러 클라우드를 공유하여 사용하는 분산형 클라우드도 블록체인에서 사용되고 있음)

#### [`Public`] 
##### - `서비스 유지`를 위한 모든 인프라와 IT기술을 클라우드에서 사용
##### - AWS, GCP, Azure와 같은 외부 클라우드 컴퓨팅 사업자가 IT자원을 소유하고 인터넷을 통해 제공
##### - IT관리 `인력이나 인프라 구축 비용이 없는 경우`에 유용

#### [`Private`] 
##### - 고객이 `자체 데이터 센터에서 직접 클라우드 서비스를 구축`하는 형태
##### - 보안이 좋고 customization이 가능하며 판매자는 IT기술만 패키지 형태로 판매하기도 함

#### [`Hybrid`] 
##### - 고객의 핵심 시스템은 내부에 두면서도 외부의 클라우드를 활용하는 형태
##### - 퍼블릭의 경제성과 프라이빗의 보안성을 모두 고려

### 1-3) 클라우드 서비스 `제공 모델`
#### `클라우드 서비스 제공 방식`에 따라 세 가지 형태로 구분
#### `IaaS`, `PaaS`, `SaaS` (`On-Premises` : 하드웨어, 클라우드, 소프트웨어어플리케이션 등을 모두 관리하는 모델)

#### [`IaaS`] : 인프라 측면만 빌리고 나머지는 직접 관리
##### - `App`, `Data`, `Runtime`, `Middleware`, `OS` 직접 관리
##### - `Virtualization`, `Servers`, `Storage`, `Networking` 같은 인프라 측면은 빌림

#### [`PaaS`] : 인프라 측면만 빌리고 나머지는 직접 관리
##### - `App`, `Data` 직접 관리
##### - `Runtime`, `Middleware`, `OS`, `Virtualization`, `Servers`, `Storage`, `Networking` 플랫폼, 인프라 측면을 빌림

#### [`SaaS`] : 인프라 측면만 빌리고 나머지는 직접 관리
##### - `App`, `Data`, `Runtime`, `Middleware`, `OS`, `Virtualization`, `Servers`, `Storage`, `Networking` 서비스, 플랫폼, 인프라 측면을 모두 빌림
##### - ex) office365

- 다양한 사업자들 : AWS, GCP, Azure, NCP
- 이번 실습에서는 `Amazon EC2`를 사용할 예정

----
## 2. 실습 : AWS & 실습 환경 세팅

### 2-1) AWS 계정 가입 (https://aws.amazon.com/ko/)
- 실습 환경 세팅은 영상 및 학습 보고서 참고하기 

----
## 3. API to serve ML model
- 본 실습에서는 `AWS EC2`와 `Python Flask`를 기반으로 모델 학습 및 추론을 요청/응답하는 `API 서버 개발`을 만드는 것이 목표

![api_sever](./pic/api_sever.png)
- API Server with DL

### 3-1) `Interface`
- 상호 `합의된 매뉴얼`에 따라 적절한 `입력에 따른 출력이 제공`하는 제품
- ex) 마우스, 키보드 : 입력을 위한 인터페이스
- ex) 모니터, 프린터 : 정보를 받는 출력을 위한 인터페이스

### 3-2) `API` (Application Programming Interface)
- `노드`와 `노드`(기계와 기계, sw와 sw)간 데이터를 주고 받는 `커뮤니케이션을 위한 인터페이스` (합의된 입력에 따른 출력)

### 3-3) `RESTful API`
- REST 아키텍쳐를 따르는 API. (`HTTP URI`, `HTTP Method`를 통해 필요한 연산을 요청, 반황하는 API)
- 일반적으로 데이터 값을 담아 요청하고 모델이 추론한 결과에 대한 return을 `json 형태로 반환`하도록 설계
- RESTful API는 `요청 메시지만 봐도 어떤 내용인지` 알 수 있도록 표현됨

### 3-4) `Practical process of ML` 
- `문제 정의` -> `Data 준비`, `Build and Train Model` ->  `Deploy Model`, `Monitor/Scale in app`
- 실습에서는 저장된 `Model File을 불러와서 Run/Predict` 하는 것이 목표

### 3-5) `Model Serving`
- `Model Training` -> `Serializing Model`(학습된 모델을 저장) -> `Serving Model`
- 이때 주의할 점은 `모든 과정이 연속성이 유지`되어야한다. (데이터 처리 방식과 데이터 분포가 서로 다르다면 기능이 떨어질 수 있다.)
- 상황에 맞는 다양한 Framework를 통해 serving을 해야함.
- framework ex) `Tensor Flow serving`, `TorchServe`, `Tensor RT` 등

### 3-6) `Serialization`
- 학습한 모델을 저장하고 불러오는 것
- `Serialization` : Memory에서 Disk로 저장
- `De-serialization` : Disk에 있는 정보를 Memory로 불러 읽어옴
- 두 방식은 동일해야함

### 3-7) `Skeleton of handler to serve model`
- 사람이 `모델의 결과 값을 잘 읽기 위해` 데이터를 `레이블링` (후처리) 하는 과정

----
## 4. kdt ai aws 실습

### 4-1) 실습 환경 세팅

In [ ]:
# aws EC2에서 아나콘다 가상환경 실행
%conda activate pytorch_p39

# 가상환경이 제대로 실행됐는지 확인
%python
%import torch
%torch.__version__
%exit()

# template 소스코드 다운로드
%git clone https://github.com/sackoh/kdt-ai-aws

# 다운받은 파일의 경로로 이동
%ls
%cd ./kdt-ai-aws/

# 필요 라이브러리 설치
%pip install -r requirements.txt

### 4-2) Model Training
- Data preprocessing
- Model fitting
- Evaluation